In [19]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import glob
import gzip
import os

In [2]:
gemma_dir = "/u/home/j/jzou1115/project-zarlab/CFW/MegaAnalysis_CFW_2019_Summary_Statistics/COMBINED"
gemma_files = glob.glob(gemma_dir+"/*")
print(len(gemma_files))

68


# snp files
- https://genome.sph.umich.edu/wiki/LocusZoom_Standalone#METAL_formatted_file
- MarkerName	P-value

In [6]:
header = ["MarkerName", "P-value"]

In [7]:
for file in gemma_files:
    f = pd.read_csv(file, sep="\t")
    out = f.loc[:,["rs", "p_wald"]]
    out.columns = header
    outfile = file[:-7]+"_locuszoom.txt.gz"
    out.to_csv(outfile, header=True, index=False, sep="\t")

# ld files

User-supplied LD
If new_fugue is installed, LocusZoom will automatically compute LD between the reference SNP and all other SNPs within each region to be plotted. However, you may wish to provide your own file with LD information. This can be done with the --ld option, which requires a file of the following format:


Columns
1. snp1: Any SNP in your plotting region.
2. snp2: Should always be the reference SNP in the region.
3. dprime: D' between snp2 (reference SNP) and snp1.
4. rsquare: r2 between snp2 (reference SNP) and snp1.

The dprime column can be all missing if it is not known. Rsquare must be present, and must be valid data.

The file should be whitespace delimited, and the header (column names shown above) must exist.

In [2]:
ci = pd.read_table("/u/home/j/jzou1115/project-zarlab/CFW/confidence_intervals/output2/confidence_intervals/CI_pruned.txt")
ci = ci.dropna(subset=["from.bp", "to.bp"])
ci_95 = ci[ci["quantile"]==.95]
ci_95.head()

,phenotype,logP,chr,bp,quantile,delta.logP,from.bp,to.bp,width.bp,n.snps,fine.bp,fine.logP,fine.from.bp,fine.to.bp,fine.width.bp,fine.n.snps,fine.CI.logP,fine.CI.bp,fine.CI.from.bp,fine.CI.to.bp
4,activity30,8.494170,13,8139647,0.95,1.535661,7101524.0,8211378.0,1109855.0,4883.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,activity.begin,6.295724,11,96964818,0.95,1.599580,92663897.0,97100561.0,4436665.0,4598.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14,activity.begin,6.030024,2,153696216,0.95,1.514107,153342082.0,153879574.0,537493.0,312.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19,activity.end,7.969123,11,96653605,0.95,1.598966,96583134.0,96912135.0,329002.0,1198.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24,activity.end,6.054185,13,10746304,0.95,1.604569,5954638.0,10765251.0,4810614.0,27066.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [30]:
ci_95.shape

(32, 20)

In [3]:
g_dir="/u/home/j/jzou1115/project-zarlab/CFW/MegaAnalysis_CFW_2019/genotypes/oxinfo90"
outdir="/u/home/j/jzou1115/project-zarlab/CFW/confidence_intervals/output2/ld"

In [28]:
for i in ci_95.index:
    pheno = ci_95.loc[i, "phenotype"]
    chrm = "chr"+str(ci_95.loc[i, "chr"])  
    bp = str(ci_95.loc[i, "bp"])
    qtl_id = "_".join([pheno, chrm, bp])
    print(qtl_id)
    if os.path.isfile(outdir+"/"+qtl_id+"_r2.txt"):
        continue
        
    start_ci = ci_95.loc[i, "from.bp"]
    end_ci = ci_95.loc[i, "to.bp"]
    
    locus = []
    dosages = []
    
    with gzip.open(g_dir+"/combined."+str(chrm)+".oxinfo90.dosages.gz",'rt') as f:
        for line in f:

            tokens = line.split("\t")
            snp_tokens = tokens[0].split("_")
            pos = int(snp_tokens[1])
            if pos >= start_ci and pos <= end_ci:
                locus.append(tokens[0])
            
                dos = tokens[3:]
                dos = [float(d) for d in dos]
                dosages.append(dos)
            
        
    dosages = pd.DataFrame(dosages)
    dosages.index = locus

    ref_snp = chrm+"_"+str(bp)

    out = open(outdir+"/"+qtl_id+"_r2.txt", "w")
    out.write("snp1\tsnp2\trsquare\n")
    for snp in dosages.index:
        if snp != ref_snp:
            temp = dosages.loc[[ref_snp, snp], :]
            temp = temp.transpose()
            r = temp.corr()
            r = r.loc[ref_snp,snp]
            r2 = r*r
                
            out.write(snp+"\t"+ref_snp+"\t"+str(r2)+"\n")
    out.close()
        
    

activity30_chr13_8139647
activity.begin_chr11_96964818
activity.begin_chr2_153696216
activity.end_chr11_96653605
activity.end_chr13_10746304
activity.middle_chr9_81879809
activity.total_chr11_96653605
activity.total_chr13_10144490
basal.activity_chr11_96816215
bmd_chr11_94772022
bmd_chr15_86382123
bmd_chr5_21721308
bmd.a_chr11_94882898
bmd.a_chr11_96913720
EDL_chr13_113025220


KeyboardInterrupt: 

In [23]:
    i = 4
    pheno = ci_95.loc[i, "phenotype"]
    chrm = "chr"+str(ci_95.loc[i, "chr"])  
    bp = str(ci_95.loc[i, "bp"])
    qtl_id = "_".join([pheno, chrm, bp])
    print(qtl_id)
    
    start_ci = ci_95.loc[i, "from.bp"]
    end_ci = ci_95.loc[i, "to.bp"]

activity30_chr13_8139647


In [24]:
    locus = []
    dosages = []
    
    with gzip.open(g_dir+"/combined."+str(chrm)+".oxinfo90.dosages.gz",'rt') as f:
        for line in f:

            tokens = line.split("\t")
            snp_tokens = tokens[0].split("_")
            pos = int(snp_tokens[1])
            if pos >= start_ci and pos <= end_ci:
                locus.append(tokens[0])
            
                dos = tokens[3:]
                dos = [float(d) for d in dos]
                dosages.append(dos)
            
        
    print(len(locus)) 

7570


In [25]:
    dosages = pd.DataFrame(dosages)
    dosages.index = locus

In [26]:
dosages.shape

(7570, 3234)

In [18]:
        out = open(outdir+"/"+qtl_id+"_r2.txt", "w")
        out.write("snp1\tsnp2\trsquare\n")
        for snp in dosages.index:
            if snp != ref_snp:
                temp = dosages.loc[[ref_snp, snp], :]
                temp = temp.transpose()
                r = temp.corr()
                r = r.loc[ref_snp,snp]
                r2 = r*r
                
                out.write(snp+"\t"+ref_snp+"\t"+str(r2)+"\n")
        out.close()